# Assignment 12

### 1.复习上课内容

### 2.回答以下理论问题

#### 1. 请写一下TF-IDF的计算公式

ans:一个词的TF-IDF = TF * IDF

TF = count(t,d) 在一个句子d中t这个词出现的次数

IDF = log10(N/dft) N为文档总数，dft为出现这个词的次数

#### 2. LDA算法的基本假设是什么？

ans:1. 每个文档的土体服从一个分布

2. 每个主题的词服从一个分布

#### 3. 在TextRank算法中构建图的权重是如何得到的？

ans:1.按照节点个数N平均初始化一个节点的权重1/N

2. 按照textrank的计算公式，在循环中不停的计算每个节点权重

3. 直到计算结果收敛，就到了图中每个节点的权重

#### 4. 什么是命名实体识别？ 有什么应用场景？

ans: NER命名实体识别是把一句话中的名词按照类型标注出来，名字、地点、品牌等等。

用于知识图谱的构建、信息的挖掘

#### 5.NLP主要有哪几类任务 ？

ans:大体分类分类任务和生成任务

1. 分类任务：文本分类、情感分析

2. 生成任务：机器翻译、问答系统、

### 3.实践题

#### 3.1 手动实现TextRank算法 (在新闻数据中随机提取100条新闻训练词向量和做做法测试）

 提示：
 1. 确定窗口，建立图链接。   
 2. 通过词向量相似度确定图上边的权重
 3. 根据公式实现算法迭代(d=0.85)

In [88]:
import pandas as pd
import jieba
import re
import numpy as np
from gensim.models import Word2Vec
from functools import lru_cache#缓存

#系数
d = 0.85

#词向量模型
modefile = 'D:/Repos/NlpProject_01/Jeff/File/06wiki_news_word2vec_model'
wordmodel = Word2Vec.load(modefile)

In [136]:
def cut(string): return list(jieba.cut(string))

def getStopwords():
    stopwords = [line.strip() for line in open('baidu_stopwords.txt',encoding='UTF-8').readlines()]
    return stopwords

def token(string):
    # we will learn the regular expression next course.
    return re.findall('\w+', string)

@lru_cache(maxsize=2**10)#缓存
def cosinewords(word1,word2):
    if word1 not in wordmodel or word2 not in wordmodel:
        return 1e-3
    return cosine(wordmodel[word1],wordmodel[word2])

def cosine(vec1,vec2):
    n1 = np.squeeze(np.asarray(vec1))
    n2 = np.squeeze(np.asarray(vec2))
    cos = np.dot(n1, n2) / (np.linalg.norm(n1) * np.linalg.norm(n2))
    return cos

In [8]:
#读取文章
content = pd.read_csv('D:/Repos/MyPython/Nlp/sqlResult_1558435.csv', encoding='gb18030')
content[u'content'] = content[u'content'].astype(str)
content[u'content'] = content[u'content'].apply(lambda x : x.replace('\r',''))
content[u'content'] = content[u'content'].apply(lambda x : x.replace('\n',''))
content[u'content'] = content[u'content'].apply(lambda x : x.replace('\u3000',''))
content[u'content'] = content[u'content'].apply(lambda x : x.replace('\\n',''))
content[u'content'] = content[u'content'].apply(lambda x : x.replace(' ',''))
articles = content['content'].tolist()

In [9]:
#停用词表
stopwords = getStopwords()

In [44]:
#初始化数据
def initdata(article):
    article = ''.join(re.findall('[\u4e00-\u9fa5]+',article))#只保留中文
    TOKEN = cut(''.join(token(article)))
    return list(filter(lambda s: s not in stopwords, TOKEN))

In [117]:
#获得窗口数据
words = initdata(articles[0])
window = {}
for i in range(len(words) - 1):
    if words[i] not in window:
        if i - 1 >= 0: 
            value = [words[i-1],words[i+1]]
        else:
            value = [words[i+1]]
        window[words[i]] = value
    else :
        value = window[words[i]]
        if words[i-1] not in value:
            value.append(words[i-1])
        if words[i+1] not in value:
            value.append(words[i+1])
        window[words[i]] = value

In [119]:
' '.join(initdata(articles[0]))

'本周 月 日 小米 手机 等款 机型 外 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 精力 系统优化 工作 有人 猜测 精力 用到 研发 之中 去年 月 发布 距今已有 一年 有余 更新换代 确切 信息 等待 官方消息'

In [118]:
window

{'本周': ['月'],
 '月': ['本周', '日', '去年', '发布'],
 '日': ['月', '小米'],
 '小米': ['日', '手机'],
 '手机': ['小米', '等款'],
 '等款': ['手机', '机型'],
 '机型': ['等款', '外', '已'],
 '外': ['机型', '机型'],
 '已': ['机型', '暂停'],
 '暂停': ['已', '更新'],
 '更新': ['暂停', '发布'],
 '发布': ['更新', '含', '月', '距今已有'],
 '含': ['发布', '开发'],
 '开发': ['含', '版'],
 '版': ['开发', '体验版'],
 '体验版': ['版', '内测'],
 '内测': ['体验版', '稳定版'],
 '稳定版': ['内测', '暂不受'],
 '暂不受': ['稳定版', '影响'],
 '影响': ['暂不受', '确保'],
 '确保': ['影响', '工程师'],
 '工程师': ['确保', '精力'],
 '精力': ['工程师', '系统优化', '猜测', '用到'],
 '系统优化': ['精力', '工作'],
 '工作': ['系统优化', '有人'],
 '有人': ['工作', '猜测'],
 '猜测': ['有人', '精力'],
 '用到': ['精力', '研发'],
 '研发': ['用到', '之中'],
 '之中': ['研发', '去年'],
 '去年': ['之中', '月'],
 '距今已有': ['发布', '一年'],
 '一年': ['距今已有', '有余'],
 '有余': ['一年', '更新换代'],
 '更新换代': ['有余', '确切'],
 '确切': ['更新换代', '信息'],
 '信息': ['确切', '等待'],
 '等待': ['信息', '官方消息']}

In [160]:
Score = {}
#初始化得分为1/n
for k,v in window.items():
    Score[k] = 1

In [161]:
#循环获得text-rank
newSocre = {}
for i in range(500) :
    for k,v in window.items(): 

        #print('clc {}'.format(k))
        wordsin = v#wordin列表
        clcscore = 1- d#初始化
        for i in range(len(wordsin)):
            #print(wordsin[i])
            outvalue = 0
            wordsout = window[words[i]]#wordout列表
            for j in range(len(wordsout)):
                #print('word1 {}'.format(wordsin[i]))
                #print('word2 {}'.format(wordsout[j]))
                outvalue = outvalue + cosinewords(wordsin[i],wordsout[j])
            clcscore = clcscore + d * cosinewords(k,wordsin[i])/outvalue * Score[k]
        newSocre[k] = clcscore

    for k,v in window.items():
        Score[k] = newSocre[k]

#print(Score)
L = sorted(Score.items(), key=lambda x: x[1], reverse=True)
#得到前10个关键词
print(L[:10])

[('内测', 1.057255110246802e+253), ('手机', 5.553954849080865e+250), ('精力', 8.348804817634573e+231), ('发布', 1.8569407043385673e+230), ('信息', 2.8959903195692976e+224), ('稳定版', 2.0831721855507155e+213), ('体验版', 2.361712679136341e+211), ('开发', 4.29705252694075e+194), ('猜测', 1.8139335117226655e+183), ('用到', 6.626059461130083e+179)]


#### 选做 1.  提取新闻人物里的对话。(使用以上提取小数据即可）

提示：    
1.寻找预料里具有表示说的意思。    
2.使用语法分析提取句子结构。    
3.检测谓语是否有表示说的意思。

#### 选择2. ： 电影评论分类。

在这个作业中你要完成一个电影评论分类任务。

1.数据获取。（采用爬虫技术爬取相关网页上的电影评论数据，例如猫眼电影评论，豆瓣电影评论）

2.把所获得数据分解为训练集，验证集和测试集。

3.选用相应算法构建模型，并测试。

#### 选择3：文章自动续写

在这个作业中你要完成一个文章自动续写的模型。

1.数据获取。（根据你的兴趣采用爬虫技术爬去相关网站上的文本数据内容：比如故事网站，小说网站等）

2.选取模型，并训练。

3.展示一些你模型的输出例子。